In [8]:
from data_reader import SimulationDataset, generate_metadata
import json
import torch
import lightning as L

# INPUT_SEQUENCE_LENGTH = 6
DEVICE = 'cpu'

ds = SimulationDataset(device=DEVICE, data_dir='data/train', window_size=6)
metadata = generate_metadata(ds)

# with open('new_metadata.json', 'rt') as f:
#     metadata = json.loads(f.read())

# num_steps = metadata['sequence_length'] - INPUT_SEQUENCE_LENGTH


In [10]:
from learned_simulator import Simulator

simulator = Simulator(
    particle_dimension=2,
    node_in=30,
    edge_in=3,
    latent_dim=128,
    num_message_passing_steps=10,
    mlp_num_layers=2,
    mlp_hidden_dim=128,
    num_particle_types=9,
    particle_type_embedding_size=16,
    metadata=metadata,
    strategy='baseline',
    device=DEVICE,
)

In [11]:
trainer = L.Trainer(max_steps=100, accelerator=DEVICE)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/rafajm/.pyenv/versions/3.9.2/envs/simulation/lib/python3.9/site-packages/lightning/pytorch/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.


In [12]:
import torch.utils.data as data

train_set_size = int(len(ds) * 0.8)
valid_set_size = len(ds) - train_set_size
seed = torch.Generator().manual_seed(42)
train_set, val_set = data.random_split(ds, [train_set_size, valid_set_size], generator=seed)
train_loader = data.DataLoader(train_set, batch_size=2, shuffle=True, num_workers=0)
val_loader = data.DataLoader(val_set, batch_size=2, shuffle=False, num_workers=0)


trainer.fit(model=simulator, train_dataloaders=train_loader, val_dataloaders=val_loader)



  | Name                     | Type                | Params | Mode 
-------------------------------------------------------------------------
0 | _particle_type_embedding | Embedding           | 144    | train
1 | _graph_network           | EncodeProcessDecode | 1.6 M  | train
-------------------------------------------------------------------------
1.6 M     Trainable params
0         Non-trainable params
1.6 M     Total params
6.367     Total estimated model params size (MB)


/Users/rafajm/.pyenv/versions/3.9.2/envs/simulation/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
/Users/rafajm/.pyenv/versions/3.9.2/envs/simulation/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Epoch 0:   0%|          | 15/27418 [00:01<55:40,  8.20it/s, v_num=14, training_loss_step=11.40] 

/Users/rafajm/.pyenv/versions/3.9.2/envs/simulation/lib/python3.9/site-packages/lightning/pytorch/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...
